In [1]:
import warnings
warnings.filterwarnings('ignore')
from csrank.experiments import DBConnector, get_dataset_reader
import os
import inspect
from csrank.util import setup_logging
from csrank.tensorflow_util import configure_numpy_keras
import logging
from result_script import get_letor_string, get_dataset_name
import pandas as pd
import numpy as np

Using TensorFlow backend.
/home/prithagupta/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
logpath = os.path.join(os.getcwd(), 'logs' ,"configtables2.log")
! rm $logpath
setup_logging(log_path=logpath)
logger = logging.getLogger('Experiment')
DIR_PATH = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
config_file_path = os.path.join(DIR_PATH, 'config', 'clusterdb.json')

rm: cannot remove '/home/prithagupta/cs-ranking/experiments/logs/configtables2.log': No such file or directory


In [3]:
def get_jobs(schema = "choice_functions", learner= "glm_choice"):
    select_jobs = "SELECT dataset_params, dataset from {0} where learner=\'{1}\' and fold_id=0 and (dataset_params->>'n_objects'='{2}' or dataset_params->>'n_objects'='30')"
    self = DBConnector(config_file_path=config_file_path, is_gpu=False, schema=schema)
    self.init_connection()
    avail_jobs = "{}.avail_jobs".format(schema)
    problem = ''
    n_objects = 10
    if schema == "choice_functions":
        problem = 'Subset Choice'
    if schema == "discrete_choice":
        problem = 'Discrete Choice'
    if schema == "object_ranking":
        problem = 'Object Ranking'
        n_objects = 5
    select_st = select_jobs.format(avail_jobs, learner, n_objects)
    print(select_st)
    self.cursor_db.execute(select_st)
    jobs = self.cursor_db.fetchall()
    
    return jobs, problem

In [4]:
def get_length(xdict):
    l = 0
    for k in xdict.keys():
        l+=xdict[k].shape[0]
    return l
def get_set_range(xdict):
    keys = np.array(list(xdict.keys()))
    mmax, mmin = np.max(keys), np.min(keys)
    return mmax, mmin

In [5]:
def get_letor_lengths(name, params):
    dr = get_dataset_reader(name, params)
    d = dr.__dict__
    if "letor" in job['dataset']:
        a = []
        mmax, mmin = 0, np.inf
        for fold_id in range(5):
            params['fold_id'] = fold_id
            dr = get_dataset_reader(name, params)
            train_n, test_n = get_length(dr.Y_train),  get_length(dr.Y_test)
            a.append([train_n, test_n])
            val = list(dr.X_train.values())[0].shape[-1]
            c,d = get_set_range(dr.X_train)
            mmax = max(mmax, c)
            mmin = min(mmax, d)
        a = np.array(a)
        arr1 = [str([i,j]) for i,j in zip(np.min(a, axis=0), np.max(a, axis=0))]
        arr1.append(val)
        arr1.append(str([mmax, mmin]))
        return arr1
    elif "exp" in job['dataset']:
        arr1 = []
        params['fold_id'] = 0
        dr = get_dataset_reader(name, params)
        dr.split_dictionaries()
        train_n, test_n = get_length(dr.Y_train),  get_length(dr.Y_test)
        arr1.extend([train_n, test_n])
        val = list(dr.X_train.values())[0].shape[-1]
        arr1.append(val)
        mmax, mmin = get_set_range(dr.X_train)
        arr1.append(str([mmax, mmin]))
        return arr1
    else:
        return [d['n_train_instances'], d['n_test_instances'], d['n_features'], d['n_objects']]

In [6]:
import copy
columns = ['Problem', 'Dataset', "#Train", '#Test', '#Features', '|Q|']
dataFrame = None 
for s, l  in zip(['discrete_choice', "choice_functions", 'object_ranking'], ['ranksvm_dc', 'ranksvm_choice', 'ranksvm']):
    jobs, problem = get_jobs(s, l)
    data = []
    for job in jobs:
        job = dict(job)
        condition = [c in job['dataset_params'].get('dataset_type', '0') for c in ['critique', 'largest', 'nearest_neighbour_medoid']]
        condition = np.any(condition)
        if condition:
            continue
        vals = [problem]
        if "letor" in job['dataset'] or "exp" in job['dataset']:
            d = get_letor_string(job['dataset_params']).title()
        elif "sushi" in job['dataset']:
            d = job['dataset'].title()
        else:
            d = job['dataset_params']['dataset_type'].title()
        vals.append(get_dataset_name(d))
        vals.extend(get_letor_lengths(job['dataset'], job['dataset_params']))
        data.append(vals)
    df = pd.DataFrame(data, columns=columns)
    df = df.sort_values('Dataset')
    if dataFrame is None:
        dataFrame = copy.copy(df)
    else:
        dataFrame = dataFrame.append(df, ignore_index=True)

SELECT dataset_params, dataset from discrete_choice.avail_jobs where learner='ranksvm_dc' and fold_id=0 and (dataset_params->>'n_objects'='10' or dataset_params->>'n_objects'='30')
SELECT dataset_params, dataset from choice_functions.avail_jobs where learner='ranksvm_choice' and fold_id=0 and (dataset_params->>'n_objects'='10' or dataset_params->>'n_objects'='30')
SELECT dataset_params, dataset from object_ranking.avail_jobs where learner='ranksvm' and fold_id=0 and (dataset_params->>'n_objects'='5' or dataset_params->>'n_objects'='30')


In [7]:
dataFrame.replace(to_replace=r'10 Objects', value='', regex=True, inplace=True)
dataFrame = dataFrame.sort_values(by=[columns[0], columns[1]], ascending=[True, True])
dataFrame

,Problem,Dataset,#Train,#Test,#Features,|Q|
0,Discrete Choice,Expedia,78041,312229,17,10
1,Discrete Choice,Hypervolume,10000,100000,2,10
2,Discrete Choice,LETOR MQ2007,"[1353, 1356]","[336, 339]",46,10
3,Discrete Choice,LETOR MQ2008,"[627, 628]","[156, 157]",46,10
4,Discrete Choice,Medoid,10000,100000,5,10
5,Discrete Choice,Mode,10000,100000,128,10
6,Discrete Choice,Tag Genome Dissimilar,10000,100000,1128,10
7,Discrete Choice,Tag Genome Similar,10000,100000,1128,10
8,Discrete Choice,Unique,10000,100000,128,10
15,Object Ranking,Hypervolume,10000,100000,2,5


In [8]:
df = copy.copy(dataFrame)
del df['Problem']
df.insert(0, 'Problem', '')
latex_code = df.to_latex(index = False)
print(latex_code)

\begin{tabular}{llllrr}
\toprule
Problem &                 Dataset &        \#Train &       \#Test &  \#Features &  |Q| \\
\midrule
        &                Expedia  &         78041 &      312229 &         17 &   10 \\
        &             Hypervolume &         10000 &      100000 &          2 &   10 \\
        &           LETOR MQ2007  &  [1353, 1356] &  [336, 339] &         46 &   10 \\
        &           LETOR MQ2008  &    [627, 628] &  [156, 157] &         46 &   10 \\
        &                  Medoid &         10000 &      100000 &          5 &   10 \\
        &                    Mode &         10000 &      100000 &        128 &   10 \\
        &   Tag Genome Dissimilar &         10000 &      100000 &       1128 &   10 \\
        &      Tag Genome Similar &         10000 &      100000 &       1128 &   10 \\
        &                  Unique &         10000 &      100000 &        128 &   10 \\
        &             Hypervolume &         10000 &      100000 &          2 &    5 \